<a href="https://colab.research.google.com/github/VivekDubey18/Digit_Recognizing_using_Tensorflow/blob/main/Digit_Recognizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TensorFlow/Keras version
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical


# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()



# Normalize input images i.e range is converted [0-255] --> [0-1] to improve performance and stability
x_train = x_train / 255.0
x_test = x_test / 255.0


#Converts the integer labels e.g. 3-> into  --> one-hot vectors  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]) to work efficiently with categorical_crossentropy
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),   # Conv Layer 1  (1st layer after input layer)
    MaxPooling2D(pool_size=(2, 2)),                                   # Max Pooling
    Conv2D(64, (3, 3), activation='relu'),                            # Conv Layer 2
    MaxPooling2D(pool_size=(2, 2)),                                   # Max Pooling
    Flatten(),                                                        # Flatten to 1D
    Dense(128, activation='relu'),                                    # Fully connected layer  (total dense layer = 1 that contain 128 neurons)
    Dropout(0.5),                                                     # Dropout for regularization
    Dense(10, activation='softmax')                                   # Output Layer (10 nerons for 10 classes 0-9)
])


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # to calculate loss function
              metrics=['accuracy'])                  # to calculate accuracy



#Define callbacks
#it saves the best performing model ignoring the model where accuracy decreases
checkpoint = ModelCheckpoint(
    'best_model.h5', monitor='val_accuracy', save_best_only=True,
    mode='max', verbose=1
)


#to handle overfitting it monitor validation_loss ....ans if validation loss continues for 3 epochs it will terminate and saves the model where the best weights are present
early_stop = EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True, verbose=1
)

#Train the model with callbacks
history = model.fit(
    x_train, y_train,
    epochs=8,
    batch_size=32,
    validation_split=0.1,
    callbacks=[checkpoint, early_stop]

In [ ]:
#Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy : {test_acc:.4f}")